In [1]:
import requests
import defs
import pandas as pd
import pickle
from tabulate import tabulate

In [2]:
session = requests.Session()

In [3]:
Index = ['SPX500_USD'] #The API name for retreiving S & P 500 index data from the Oanda website.

In [4]:
def fetch_candles(Index_name, count, granularity): #Function returns the data from the Oanda website based on Index type
    url = f"{defs.OANDA_URL}/instruments/{Index_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params = params, headers = defs.SECURE_HEADER)
    return response.status_code, response.json()

In [5]:
def get_price(dictionary,prices,ohlc,candle): #Function returns the dictionary for candles for mid, ask and bid prices.
    for price in prices:
        for oh in ohlc:
            dictionary[f"{price}_{oh}"] = candle[price][oh]

In [6]:
def get_candles_df(json_response):# Creates a dataframe for the candles data on a given Index type
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o','h','l','c']
    
    our_data= []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        price = get_price(new_dict,prices,ohlc,candle)
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [7]:
def save_file(candles_df, Index_type, granularity):#Saves Files to a pickle document
    candles_df.to_pickle(f"{Index_type}_{granularity}.pkl")

In [8]:
def create_data(Index_type, count, granularity):
    code, json_data = fetch_candles(Index_type,count,granularity)
    if code != 200:
        print(pair,"Error")
        return
    df = get_candles_df(json_data)
    print(df)
    print(f"{Index_type} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df,Index_type,granularity)

In [17]:
try:
    create_data(Index[0],4900,"D")
except:
    print("Caught it!")

                                time  volume   mid_o   mid_h   mid_l   mid_c  \
0     2003-03-23T22:00:00.000000000Z     823   882.7   883.2   861.9   864.4   
1     2003-03-24T22:00:00.000000000Z     815   863.4   880.4   857.6   874.2   
2     2003-03-25T22:00:00.000000000Z     791   874.4   876.7   866.6   869.6   
3     2003-03-26T22:00:00.000000000Z     770   867.6   874.9   858.1   868.9   
4     2003-03-27T22:00:00.000000000Z     787   870.1   870.4   860.8   863.9   
...                              ...     ...     ...     ...     ...     ...   
4718  2021-04-28T21:00:00.000000000Z   72763  4194.0  4220.0  4177.2  4209.2   
4719  2021-04-29T21:00:00.000000000Z   85459  4209.3  4210.1  4176.4  4189.0   
4720  2021-05-02T21:00:00.000000000Z   59419  4192.4  4211.4  4190.2  4192.6   
4721  2021-05-03T21:00:00.000000000Z  112144  4192.8  4194.3  4129.6  4167.0   
4722  2021-05-04T21:00:00.000000000Z   60654  4167.8  4188.8  4162.0  4167.0   

       bid_o   bid_h   bid_l   bid_c   

In [19]:
with open("SPX500_USD_D.pkl", "rb") as f:
    object = pickle.load(f)
    
df = pd.DataFrame(object)
df.to_csv(r'SPX500_Data.csv')